In [5]:
import os
import time
import subprocess

# === CONFIGURATION ===
WATCH_DIR = r"C:\Users\lokes\OneDrive\Documents\Projects\statistics-notebooks"
BRANCH = "main"
INTERVAL = 5  # in seconds

def git_run(cmd, cwd):
    """Run git commands safely."""
    return subprocess.run(cmd, cwd=cwd, capture_output=True, text=True)

def watch_and_sync(directory, branch="main", interval=5):
    print(f"Watching folder: {directory}")
    
    # Change working directory to repo
    os.chdir(directory)

    # Ensure this is a Git repository
    if not os.path.exists(os.path.join(directory, ".git")):
        print("Initializing Git repository...")
        git_run(["git", "init"], directory)
        git_run(["git", "branch", "-M", branch], directory)
        print("Git repo initialized.\n")

    last_snapshot = {}

    while True:
        current_snapshot = {}

        # Track the modified time of each file
        for root, dirs, files in os.walk(directory):
            for filename in files:
                if filename.endswith(".ipynb") or filename.endswith(".py"):
                    filepath = os.path.join(root, filename)
                    current_snapshot[filepath] = os.path.getmtime(filepath)

        # Detect changes
        changed_files = [
            f for f in current_snapshot
            if f not in last_snapshot or current_snapshot[f] != last_snapshot[f]
        ]

        if changed_files:
            print("Changes detected:", changed_files)
            git_ru_


In [6]:
watch_and_sync(WATCH_DIR, branch=BRANCH, interval=INTERVAL)

Watching folder: C:\Users\lokes\OneDrive\Documents\Projects\statistics-notebooks
Changes detected: ['C:\\Users\\lokes\\OneDrive\\Documents\\Projects\\statistics-notebooks\\auto_sync_notebook.ipynb', 'C:\\Users\\lokes\\OneDrive\\Documents\\Projects\\statistics-notebooks\\_01_pandas.ipynb', 'C:\\Users\\lokes\\OneDrive\\Documents\\Projects\\statistics-notebooks\\_02_time_series.ipynb', 'C:\\Users\\lokes\\OneDrive\\Documents\\Projects\\statistics-notebooks\\_03_numpy.ipynb', 'C:\\Users\\lokes\\OneDrive\\Documents\\Projects\\statistics-notebooks\\.ipynb_checkpoints\\auto_sync_notebook-checkpoint.ipynb', 'C:\\Users\\lokes\\OneDrive\\Documents\\Projects\\statistics-notebooks\\.ipynb_checkpoints\\_01_pandas-checkpoint.ipynb', 'C:\\Users\\lokes\\OneDrive\\Documents\\Projects\\statistics-notebooks\\.ipynb_checkpoints\\_02_time_series-checkpoint.ipynb', 'C:\\Users\\lokes\\OneDrive\\Documents\\Projects\\statistics-notebooks\\.ipynb_checkpoints\\_03_numpy-checkpoint.ipynb']


NameError: name 'git_ru_' is not defined